In [1]:
#!pip install langchain[all]
!pip install langchain[llms]
!pip install tiktoken
!pip install chromadb
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.2 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.1/794.1 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.9/257.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.9/214.9 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 92.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━

In [2]:
#https://www.youtube.com/watch?v=s5LhRdh5fu4 
# multiple documents


In [22]:
# import the libs required for chatbot with pdf files 
#load llms
from langchain.llms import OpenAI
# load data loaders
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# load embeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS, Chroma
#load chains
from langchain.chains import RetrievalQA, ChatVectorDBChain, ConversationalRetrievalChain, ConversationChain

In [4]:
# define and instantiate the llms for the activities
llm_pdfchat = OpenAI(temperature=0.6,openai_api_key=openaikey, model="text-davinci-003" )

In [5]:
# load the documnets from the source
loader = PyPDFLoader(file_path='/kaggle/input/resume-dataset/data/data/ADVOCATE/10186968.pdf')
dat = loader.load()

In [6]:
dat

[Document(page_content="CHILD FAMILY ADVOCATE\nProfessional Profile\nTalented administrative professional with background in accounting and finance. Extensive knowledge of AR/AP, Microsoft Excel and Quick\nBooks-software skills. Fearless Child Family Advocate comfortable taking a stand against threats to a child's safety or well-being. Works directly\nwith government officials, children and families to find the best care possible for every child.\nQualifications\nAttentive listener\nSensitive\nFamily maintenance\nDetail-oriented\nExcellent interpersonal skills\nTeam player\nStaff development\nRelevant Experience\nServed many program participants who went on to earn high school diplomas or GEDs.\nIncreased office organization by developing more efficient filing system and customer database protocols.\nLed support groups to help parents regain and improve their parent-child bond.\nExperience\nChild Family Advocate\n \nAugust 2013\n \nto \nCurrent\n \nCompany Name\n \nï¼\u200b \nCity\n \n

In [7]:
text_split = RecursiveCharacterTextSplitter(separators=['\n'],chunk_size=2000, chunk_overlap=50)
docs = text_split.transform_documents(dat)

In [10]:
print(docs[1].page_content)

Created training manuals targeted at resolving even the most difficult customer issues.
Developed, implemented and monitored programs to maximize customer satisfaction.
Interviewed, hired and trained new quality customer service representatives.
Developed and implemented policies, procedures and process improvement initiatives to improve retention rates and increase customer
satisfaction.
Customer Service/Banking 
November 2007
 
to 
March 2009
 
Company Name
 
ï¼​ 
City
 
, 
State
Collected customer feedback and made process changes to exceed customer satisfaction goals.
Developed process improvements to enhance efficiency and effectiveness of inter-department call center operations.
Opened new customer accounts, including checking, savings and lines of credit.
Examined checks for identification and endorsement.
Processed treasury, tax and loan payments.
Administrative Assistant
 
August 2007
 
to 
November 2007
 
Company Name
 
ï¼​ 
City
 
, 
State
Managed the receptionist area, incl

In [11]:
embed = OpenAIEmbeddings(openai_api_key=openaikey)
doc_embed = Chroma.from_documents(docs,embed)
doc_embed_faiss = FAISS.from_documents(docs,embed)

# Similarity search with the documents

In [12]:
query = "what are the core skills of the applicant?"
rslt = doc_embed.similarity_search(query)
rslt

[Document(page_content='Verified and logged in deadlines forresponding to daily inquiries.\nEducation\nl Coursework in Organizational Behavior, Compensation, Labor Relations and Recruitment \nl Continuing education in Human Services\n \nGreenville\nCollege \nl Desktop publishing workshop\n \nï¼\u200b \nCity\n \n, \nState\nSkills\nscheduling appointments\nAS 400, budgets\ncall center\nCustomer service\nfinancial, funds\nLabor Relations\nLeadership\nSales', metadata={'page': 1, 'source': '/kaggle/input/resume-dataset/data/data/ADVOCATE/10186968.pdf'}),
 Document(page_content="CHILD FAMILY ADVOCATE\nProfessional Profile\nTalented administrative professional with background in accounting and finance. Extensive knowledge of AR/AP, Microsoft Excel and Quick\nBooks-software skills. Fearless Child Family Advocate comfortable taking a stand against threats to a child's safety or well-being. Works directly\nwith government officials, children and families to find the best care possible for every

# chat bot chain

# RetrievalQA.from_llm

In [13]:
# chat bot chain
llm_chat_chain = RetrievalQA.from_llm(llm=llm_pdfchat,retriever=doc_embed.as_retriever())

# RetrievalQA.from_chain_typ

In [21]:
llm_bot_fct = RetrievalQA.from_chain_type(llm=llm_pdfchat, chain_type='refine',retriever=doc_embed.as_retriever())

In [19]:
query = "skill domain expetise"
rslt = llm_chat_chain.run(query)
rslt

' The skill domain expertise of the individual in the context is administrative, accounting and finance, customer service, call center, labor relations, leadership, sales, family maintenance, staff development, customer database protocols, support groups, government officials, parent-child bond, community education, protective placement, case management, parent education, family reunification, customer satisfaction, process improvements, opening customer accounts, treasury, tax and loan payments, receptionist duties, spreadsheets, mail distribution, sales prices, and record scanning.'

In [22]:
query = "skill domain expetise"
rslt = llm_bot_fct.run(query)
rslt

'\n\nThe skill domain expertise indicated by the context information is in the areas of customer service, scheduling appointments, AS 400, budgets, call center, financial and funds management, labor relations, leadership, AR/AP, Microsoft Excel and QuickBooks software skills, family maintenance, staff development, collaboration with community members, creating and implementing developmentally-appropriate curriculum, supervising children on field trips, protective placement, case management, parent education and family reunification activities, advising patients on community resources, making referrals, devising realistic treatment plans, communicating with public social and welfare agencies, surpassing revenue goals, leading support groups, creating training manuals targeted at resolving customer issues, developing and implementing customer satisfaction programs, interviewing and training new customer service representatives, developing process improvements to enhance efficiency and ef

In [23]:
query = "work experience of the applicant and number of years?"
rslt = llm_chat_chain.run(query)
rslt

' The applicant has work experience as a Child Family Advocate since August 2013 and as a Customer Services Team Lead from October 2010 to March 2013.'

In [25]:
query = "work experience of the applicant in number of years?"
rslt = llm_bot_fct.run(query)
rslt

'\n\nThe work experience of the applicant in number of years is approximately 8 years, from October 2010 to March 2013 and August 2013 to the present.'

In [26]:
query = "company in which he worked for longest period of time?"
rslt = llm_chat_chain.run(query)
rslt

' Company Name from August 2013 to Current.'

In [27]:
query = "company in which he worked for longest period of time?"
rslt = llm_bot_fct.run(query)
rslt

'\nCompany Name\nCity, State\nVerified and logged in deadlines for responding to daily inquiries. Education in Organizational Behavior, Compensation, Labor Relations, Recruitment, and continuing education in Human Services. Skills include scheduling appointments, AS 400, budgets, call center, customer service, financial, funds, labor relations, leadership, and sales.'

In [28]:
print(rslt)


Company Name
City, State
Verified and logged in deadlines for responding to daily inquiries. Education in Organizational Behavior, Compensation, Labor Relations, Recruitment, and continuing education in Human Services. Skills include scheduling appointments, AS 400, budgets, call center, customer service, financial, funds, labor relations, leadership, and sales.


In [29]:
query = "contact details if any?"
rslt = llm_chat_chain.run(query)
rslt

' No, the context does not provide contact details.'

In [30]:
query = "contact details if any?"
rslt = llm_bot_fct.run(query)
rslt

'\nNo, there are no contact details provided in the context information or in the professional profile of the Child Family Advocate.'

In [62]:
query = "mail id?"
rslt = llm_chat_chain.run(query)
rslt

" I don't know."

In [63]:
query = "is he a lawyer?"
rslt = llm_chat_chain.run(query)
rslt

' No, he is not a lawyer.'

In [64]:
query = "is he in legal domain or medical domain?"
rslt = llm_chat_chain.run(query)
rslt

' He is not in either the legal or medical domain. He has experience in customer service, banking, administrative assistance, and call center/phone representative/collector.'

In [65]:
query = "he mentioned as advocate, then he maight be a lawyer?"
rslt = llm_chat_chain.run(query)
rslt

' No, he is not a lawyer. He is a Child Family Advocate, which is a professional who works with children and families to find the best care possible for every child.'

# With Question Answering Chain 

In [49]:
from langchain.chains.question_answering import load_qa_chain
lod_qa_cain = load_qa_chain(llm=llm_pdfchat,chain_type='stuff')

In [50]:
query = "work experience of the applicant and number of years?"
rslt =lod_qa_cain.run(input_documents = docs,question = query)
rslt

' The applicant has 11 years of work experience. They have worked as a Child Family Advocate, Customer Services Team Lead, Customer Service/Banking, Administrative Assistant, and a Call Center/Phone Representative/Collector.'

In [51]:
query = "work experince duration year wise in Descending order?"
rslt =lod_qa_cain.run(input_documents = docs,question = query)
rslt

' August 2013 to Current as Child Family Advocate, October 2010 to March 2013 as Customer Services Team Lead, November 2007 to March 2009 as Customer Service/Banking, August 2007 to November 2007 as Administrative Assistant, August 2005 to November 2007 as Call Center/Phone Representative/Collector.'

In [53]:
query = "company in which he worked for longest period of time?"
rslt =lod_qa_cain.run(input_documents = docs,question = query)
rslt

' Company Name from August 2005 to November 2007.'

In [54]:
query = "core skill in 3 words?"
rslt =lod_qa_cain.run(input_documents = docs,question = query)
rslt

' Customer service'

In [56]:
query = "education details school in paticular?"
rslt =lod_qa_cain.run(input_documents = docs,question = query)
rslt

' Greenville College'

In [57]:
query = "date of birth and age related information?"
rslt =lod_qa_cain.run(input_documents = docs,question = query)
rslt

" I don't know."

In [58]:
query = " current location and point of contact details any ?"
rslt =lod_qa_cain.run(input_documents = docs,question = query)
rslt

" No, I don't know the current location and point of contact details."

# Handling multiple Pdf files together 

In [17]:
# load multiple pdf files first
import pandas as pd
import numpy as np
import os

In [18]:
pdf_path_list = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pdf_path_list.append(os.path.join(dirname, filename))

In [21]:
Multiple_pdf = pdf_path_list[1:5]

In [ ]:
# Load the data into dcos